# Embeddings - Vertex AI Vector Search

### Semantic Search Using Embeddings and Vertex AI Vector Search (Formerly Matching Engine)

- Semantic search is a type of search that uses the meaning of words and phrases to find relevant results.
- In this tutorial, we will demonstrate how to do semantic search with embeddings generated from the news text using vector search`
- Vertex AI Vector Search (formerly known as Matching Engine) is a vector database, which can find the most similar vectors from over a billion vectors. Matching Engine's ANN service can serve similarity-matching queries at high queries per second (QPS).

This pattern is more appropriate for larger datasets and production deployments. For demonstrating the full workflow and explanation, a small dataset is pre-processed locally and uploaded to a storage bucket in this example. In a larger or production deployment, the embedding generation and storage can be done separately and Vector Search provides db update strategies as described later in this notebook.

## Install Vertex LLM SDK

Install required libraries and initialises the Vertex AI SDK

In [4]:
# Install Required Libraries
!pip3 install "google-cloud-aiplatform>=1.25" "shapely<2.0.0"

In [1]:
# Import Vertex AI SDK
PROJECT_ID = !gcloud config get project
PROJECT_ID = PROJECT_ID.n
LOCATION = "europe-west2"
LOCATION_DEPLOY = "europe-west2" #Location to deploy GCP resources

import vertexai
from google.cloud import aiplatform

vertexai.init(project=PROJECT_ID, location=LOCATION)

2023-12-05 19:50:05.908583: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 19:50:07.091012: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-12-05 19:50:07.091154: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
PROJECT_ID

'playpen-8d8611'

## Import TextEmbeddingModel

**Available models as of Sep 2023:**
| Models | Description
| :- | :- |
| textembedding-gecko@001 | stable |
| textembedding-gecko@latest | public preview: an embeddings model with enhanced AI quality |
| textembedding-gecko-multilingual@latest | public preview: an embeddings model designed to use a wide range of non-English languages. |


Further documentation on available models can be found here: https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings#generative-ai-get-text-embedding-python

In [3]:
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

## Import Required Packages

Outputs with regard to Tensorflow can be ignored as this is caused by this notebook being CPU only, a GPU is not required for this demonstration.


In [42]:
import json
import time

import numpy as np
import pandas as pd

## Create Embedding Dataset.

The dataset is solely to demonstrate the use of the Text Embedding API with a vector database. It is not intended to be used for any other purpose, such as evaluating models. The dataset is small and does not represent a comprehensive sample of all possible text.

The following command copies the data json file from a google storage bucket, the data is stored locally within the notebook for use. 

### Load data.json as a python list

### Peek at the data

In [43]:
df = pd.read_csv('text_data.csv')
df.dropna(subset=['text'], inplace=True)
df = df[df['text'] != '']

In [11]:
cd ../help_guide

/home/jupyter/help_guide


### Get embeddings from the Google Embedding Model

The following code sends a request to the embedding model api to get the embedding vector for each entry in the dataset and stores it in a Python DataFrame.

The number of dimensions of the embedding vectors is 768 for the text-embedding gecko model.

In [44]:
def get_embedding(text):
    get_embedding.counter += 1
    try:
        if get_embedding.counter % 100 == 0:
            time.sleep(3)
        return model.get_embeddings([text])[0].values #Send request to embedding model
    except:
        return []


get_embedding.counter = 0

# This may take several minutes to complete.
df["embedding"] = df["text"].apply(lambda x: get_embedding(x))

# Convert the embeddings into a Python list 
embeddings_list = df['embedding'].values.tolist()

In [37]:
df['embedding']

0     [-0.0025650213938206434, 0.006478543858975172,...
1     [0.014412334188818932, -0.005281571764498949, ...
2     [-0.008918189443647861, -0.024692846462130547,...
3     [0.002036351477727294, -0.0008238544687628746,...
4     [0.015345461666584015, -0.011854755692183971, ...
5     [0.0017794094746932387, -0.014604448340833187,...
6     [0.027216080576181412, -0.002777953865006566, ...
7     [0.01929323561489582, -0.01205194741487503, 0....
8     [0.02833694778382778, -0.007414362393319607, -...
9     [-0.01748501881957054, -0.0207620058208704, -0...
10    [0.027162881568074226, -0.018828000873327255, ...
11    [0.020501218736171722, 0.006840361747890711, 0...
13    [0.01842471770942211, -0.04378639534115791, -0...
14    [-0.0010168530279770494, -0.010135345160961151...
15    [-0.018353095278143883, -0.008018066175282001,...
16    [-0.024011988192796707, -0.023942427709698677,...
18    [0.012686347588896751, -0.009768741205334663, ...
19    [0.021655980497598648, 0.01093670446425676

## Using Vertex AI Vector Search Approximate Nearest Neighbour (ANN) Service

Vertex AI Vector Search (formerly known as Matching Engine) is a vector database, which can find the most similar vectors from over a billion vectors. Matching Engine's ANN service can serve similarity-matching queries at high queries per second (QPS).

Further Details and Documentation: https://cloud.google.com/vertex-ai/docs/matching-engine/ann-service-overview

**Terminology**
- **Index:** A collection of vectors deployed together for similarity search. Vectors can be added to an index or removed from an index. Similarity search queries are issued to a specific index and will search over the vectors in that index.
- **Recall:** The percentage of true nearest neighbors returned by the index. For example, if a nearest neighbor query for 20 nearest neighbors returned 19 of the "ground truth" nearest neighbors, the recall is 19/20x100 = 95%.
- **Restricts:** Functionality to "restrict" searches to a subset of the index by using Boolean rules.

### Generate Embedding File


This function converts the embedding and saves it into a json file format usable with matching engine in the following format. 
- Encode the file using UTF-8.
- Make each line a valid JSON object to be interpreted as a record.
- Include in each record a field named id that requires a valid UTF-8 string that is the ID of the vector.
- Include in each record a field named embedding that requires an array of numbers. This is the feature vector.

Other file formats can be found here: https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup/format-structure#json

In [46]:
with open("datasets/vector_search_dataset.json", "w", encoding='UTF-8') as f:
    for i in range(len(embeddings_list)):
        json_object = json.dumps({
            'id': str(i),
            'embedding': embeddings_list[i]
        })
        f.write(json_object)
        f.write('\n')

### Copy Dataset to Storage Bucket
The following command copies the data file to a cloud storage bucket. Folder structure documentation: https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup/format-structure#input_directory_structure

Supported Update Methods:
- **Batch:** To update the content of an existing Index, use the IndexService.UpdateIndex method.
- **Streaming:** With Streaming Updates, you can update and query your index within a few seconds. At this time, you can't use Streaming Updates on an existing index, you must create a new index.

Documentation: https://cloud.google.com/vertex-ai/docs/matching-engine/update-rebuild-index#update_index_content_with_batch_updates

In [39]:
!mkdir -p datasets
!gsutil copy datasets/vector_search_dataset.json gs://gen-ai-{PROJECT_ID}-bucket/embeddings/vs_root/vector_search_dataset.json

Copying file://datasets/vector_search_dataset.json [Content-Type=application/json]...
/ [1 files][951.1 KiB/951.1 KiB]                                                
Operation completed over 1 objects/951.1 KiB.                                    


### Create an Index

Details on configuration parameters can be found here: https://cloud.google.com/vertex-ai/docs/matching-engine/configuring-indexes

**It can take up to 30 minutes to deploy.**


In [53]:
DIMENSIONS = 768
GS_URI = "gs://gen-ai-%s-bucket/embeddings/vs_root/" % PROJECT_ID

gen_ai_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name="Gen AI Index",
    contents_delta_uri=GS_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=5,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=10,
    leaf_nodes_to_search_percent=80,
    description="Example Index for Gen AI Playpen",
    location=LOCATION_DEPLOY
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/336030387678/locations/europe-west2/indexes/3025433787174486016/operations/5963380533638463488
MatchingEngineIndex created. Resource name: projects/336030387678/locations/europe-west2/indexes/3025433787174486016
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/336030387678/locations/europe-west2/indexes/3025433787174486016')


### Create an Index Endpoint

The following function is used to create an index endpoint, this allows for queries to be sent to the index

In [54]:
gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="Gen AI Index Endpoint",
    description="Example Index for Gen AI Playpen",
    public_endpoint_enabled=True,
    location=LOCATION_DEPLOY
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/336030387678/locations/europe-west2/indexEndpoints/8762456762491076608/operations/5463480975000338432
MatchingEngineIndexEndpoint created. Resource name: projects/336030387678/locations/europe-west2/indexEndpoints/8762456762491076608
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/336030387678/locations/europe-west2/indexEndpoints/8762456762491076608')


### Deploy the Index to the Index-Endpoint

In [56]:
gen_ai_index_endpoint = gen_ai_index_endpoint.deploy_index(
    index=gen_ai_index, deployed_index_id="gen_ai_deployed_index",
    machine_type="e2-standard-16",
    min_replica_count=1,
    max_replica_count=1
)

gen_ai_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/336030387678/locations/europe-west2/indexEndpoints/8762456762491076608


AlreadyExists: 409 There already exists a DeployedIndex with same ID "gen_ai_deployed_index" deployed or being deployed at the following IndexEndpoint: projects/336030387678/locations/europe-west2/indexEndpoints/8762456762491076608. Please use a different ID.

### Additional Functions:
Uncomment, replace the relevant project, region and ids to retrieve indexes and index endpoints. The ids can be found on the cloud console under Vertex AI and Vector Search on the left bar. This is useful as obtaining the index objects allow for functions on them (such as delete or query) if the notebook kernels etc. have been reset.

In [50]:
gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
    index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/3083417632376881152"
)

gen_ai_index = aiplatform.MatchingEngineIndex(
    index_name="projects/playpen-8d8611/locations/europe-west2/indexes/35043634600476672"
)

### Create Online Queries

Pre-define function that generates an embedding for an input prompt using the embeddings-gecko model and performs a search.

In [51]:
NUM_NEIGHBOURS = 3 #Number of neighbours from query

def search(input):
    embedding_vec =  model.get_embeddings([input])[0].values #Send request to embedding model to generate the embedding vector
    
    #find neighbours using vector search
    neighbours = gen_ai_index_endpoint.find_neighbors(
        deployed_index_id="gen_ai_deployed_index",
        queries=[embedding_vec],
        num_neighbors=NUM_NEIGHBOURS,
    )[0]
    
    for nb in neighbours:
        print("id: " + nb.id + " | text: " + df.iloc[int(nb.id)]["text"] + " | dist: " + str(nb.distance)) 

### Example Queries

If you face an openssl error, please wait up to 5 minutes for the endpoint to finish deploying.

In [52]:
search("tell me about reporting stolen card")

ServiceUnavailable: 503 failed to connect to all addresses; last error: UNAVAILABLE: ipv4:34.149.207.131:443: Socket closed

E1205 23:13:11.266752131   27614 ssl_transport_security.cc:1432]       Handshake failed with fatal error SSL_ERROR_SSL: error:10000410:SSL routines:OPENSSL_internal:SSLV3_ALERT_HANDSHAKE_FAILURE.
E1205 23:15:02.929635588   27614 ssl_transport_security.cc:1432]       Handshake failed with fatal error SSL_ERROR_SSL: error:10000410:SSL routines:OPENSSL_internal:SSLV3_ALERT_HANDSHAKE_FAILURE.
E1205 23:17:04.484823187   27614 ssl_transport_security.cc:1432]       Handshake failed with fatal error SSL_ERROR_SSL: error:10000410:SSL routines:OPENSSL_internal:SSLV3_ALERT_HANDSHAKE_FAILURE.
E1205 23:19:22.434231690   27614 ssl_transport_security.cc:1432]       Handshake failed with fatal error SSL_ERROR_SSL: error:10000410:SSL routines:OPENSSL_internal:SSLV3_ALERT_HANDSHAKE_FAILURE.


In [21]:
search("tell me about an important moment or event in your life")

id: 33 | text: Log on to Online for Business – Memorable InformationLog on securely using three characters from your memorable information, or with your Card Reader. The easiest way to log on to Online for Business is to tell us you’re on a trusted device.This means we’ll usually only ask you for three characters from your memorable information, as well as your password, each time you log on.Sometimes we may ask you to use your Card Reader, so it’s a good idea to always have one to hand.How to logonMemorable informationTrusting your deviceUsing your Card ReaderHow to set up memorable informationTo set up memorable information please log on to your account and click on the ‘Settings’ section in the top right hand corner of your Online for Business homepage.Then click on ‘change memorable information’ and follow the instructions.When you log on to your account, you will be asked to enter your user ID and your password. You will then be given the option to use either your memorable inform

### Clean Up Resources

In [8]:
gen_ai_index_endpoint.undeploy_all()
gen_ai_index_endpoint.delete() #index endpoint

NameError: name 'gen_ai_index_endpoint' is not defined

In [9]:
gen_ai_index.delete()

NameError: name 'gen_ai_index' is not defined